# Dependencies

In [1]:
#import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\maria\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


# Scrape Nasa Site for first headline

In [3]:
#URL of page to be scraped
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [4]:
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')
# Retrieve all elements that contain header information
headers = soup.find_all('div', class_='content_title')
paragraphs = soup.find_all('div', class_='article_teaser_body')      

In [5]:
news_title = headers[1].text
news_title

'Another First: Perseverance Captures the Sounds of Driving on Mars'

In [6]:
news_p = paragraphs[0].text

In [7]:
news_p

'NASA’s newest rover recorded audio of itself crunching over the surface of the Red Planet, adding a whole new dimension to Mars exploration.'

# Mars image scrape

In [8]:
#URL of page to be scraped
url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(url)

In [9]:
browser.links.find_by_partial_text('FULL IMAGE').click()

In [10]:
html = browser.html
# Parse HTML with Beautiful Soup
soup = bs(html, 'html.parser')

mar_img = soup.find_all('img', class_='fancybox-image')

In [11]:
mars_image = mar_img[0]["src"]


In [12]:
base_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/'

In [13]:
image_url = base_url + mars_image

In [14]:
image_url

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg'

# Mars Facts

In [15]:
#facts url
facts_url = 'http://space-facts.com/mars/'

# Use Panda's `read_html` to parse the url
mars_facts = pd.read_html(facts_url)

# Find the mars facts DataFrame in the list of DataFrames as assign it to `mars_df`
mars_df = mars_facts[0]

# Assign the columns `['Description', 'Value']`
mars_df.columns = ['Description','Values']

# Set the index to the `Description` column without row indexing
# mars_df.set_index('Description', inplace=True)

# Save html code to folder Assets
marshtml = mars_df.to_html(index=False)

text_file = open("marsfacts.html", "w")
text_file.write(marshtml)
text_file.close()


# Mars Hemispheres

In [16]:
# Visit hemispheres website through splinter module 
hem_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hem_url)

In [17]:
#html 
html_hem = browser.html

#parse with Beautiful Soup
soup = bs(html_hem, 'html.parser')

#get items that have mars hemisphere information
items = soup.find_all('div', class_='item')

#to collect urls
listings=[]

#base url
hem_base_url = 'https://astrogeology.usgs.gov'


In [18]:
#for loop to collect urls

for i in items:
    #store title
    title = i.find('h3').text
    
    #store link that opens full image
    partial_img_url = i.find('a', class_='itemLink product-item')['href']
    
    #visit link that contains full image
    browser.visit(hem_base_url + partial_img_url)
    
    #HTML Object of full screen
    partial_img_html = browser.html
    
    #Parse with BS
    soup_hem = bs(partial_img_html, 'html.parser')
    
    #Full image souce
    full_img = hem_base_url + soup_hem.find('img', class_='wide-image')['src']
    
    #Append into a list of dictionaries
    listings.append({"title": title, "full_img": full_img})
    
#Show URLS
listings

[{'title': 'Cerberus Hemisphere Enhanced',
  'full_img': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'full_img': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'full_img': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'full_img': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [19]:
#append the rest of mars data
listings.append({"news_title": news_title})
listings.append({"news_p": news_p})
listings.append({"image_url": image_url})

In [20]:
listings

[{'title': 'Cerberus Hemisphere Enhanced',
  'full_img': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'full_img': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'full_img': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'full_img': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'},
 {'news_title': 'Another First: Perseverance Captures the Sounds of Driving on Mars'},
 {'news_p': 'NASA’s newest rover recorded audio of itself crunching over the surface of the Red Planet, adding a whole new dimension to Mars exploration.'},
 {'image_url': 'https://data-class-jpl-space.s3.amaz

In [21]:
browser.quit()

In [23]:
listings = {}
listings['news_title'] = news_title,
listings['news_p'] = news_p,
listings['image_url'] = image_url,
listings['mars_facts'] = marshtml,
listings['hemisphere_list'] = listings

In [24]:
listings

{'news_title': ('Another First: Perseverance Captures the Sounds of Driving on Mars',),
 'news_p': ('NASA’s newest rover recorded audio of itself crunching over the surface of the Red Planet, adding a whole new dimension to Mars exploration.',),
 'image_url': ('https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg',),
 'mars_facts': ('<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Description</th>\n      <th>Values</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td